In [1]:
import qutip as qt
import numpy as np
import scqubits as scq
import matplotlib.pyplot as plt
import itertools
import warnings

In [2]:
levels = 6
fluxonium = scq.Fluxonium(EJ=8.9, EC=2.5, EL=0.5, flux=0.48, cutoff=110)
c_ops = None  # will be initialized once below

In [3]:
def init_c_ops():
    gamma_ij = {}
    for j in range(1, levels):
        for i in range(j):
            t1 = fluxonium.t1_capacitive(j, i, Q_cap=1e5)
            if t1 is not None and t1 > 0:
                rate = 1.0 / t1
                gamma_ij[(i, j)] = rate
                gamma_ij[(j, i)] = rate
    c_ops_local = []
    for (i, j), gamma in gamma_ij.items():
        cop = np.sqrt(gamma) * qt.basis(levels, i) * qt.basis(levels, j).dag()
        c_ops_local.append(cop)
    return c_ops_local

In [4]:
def evolve(omega_d, t_g):
    global c_ops
    if c_ops is None:
        c_ops = init_c_ops()

    evals, evecs = fluxonium.eigensys(evals_count=levels)
    n_op_energy_basis = qt.Qobj(fluxonium.process_op(fluxonium.n_operator(), energy_esys=(evals, evecs)))
    H0 = qt.Qobj(np.diag(evals))
    A = 0.1
    drive_op = n_op_energy_basis
    H = [H0, [A * drive_op, 'cos(wd * t)']]
    args = {'wd': omega_d}
    options = qt.Options(nsteps=1000000, store_states=True, atol=1e-10, rtol=1e-9)

    propagator = qt.propagator(H, t_g, args=args, options=options, c_ops=c_ops)
    propagator_kraus = qt.to_kraus(propagator)
    propagator_2x2 = [qt.Qobj(k.full()[:2, :2]) for k in propagator_kraus]
    p_2x2_super = qt.kraus_to_super(propagator_2x2)
    fidelity = qt.average_gate_fidelity(p_2x2_super, qt.sigmax())
    return fidelity

## Serial Execution

In [5]:
# if __name__ == "__main__":
#     evals, _ = fluxonium.eigensys(evals_count=levels)
#     omega_d_base = evals[1] - evals[0]

#     omega_d_array = np.linspace(omega_d_base - 0.005, omega_d_base + 0.005, 10)
#     peak_time_noise = 559.5559555955596  # previously determined
#     t_g_array = np.linspace(0.99 * peak_time_noise, 1.01 * peak_time_noise, 10)
#     param_pairs = list(itertools.product(omega_d_array, t_g_array))
#     print(f"Total simulations to run: {len(param_pairs)}")

#     results_flat = []
#     for (omega_d, t_g) in param_pairs:
#         print(f"Running: omega_d={omega_d:.5f}, t_g={t_g:.2f}")
#         fidelity = evolve(omega_d, t_g)
#         results_flat.append(fidelity)

#     results = np.reshape(results_flat, (len(omega_d_array), len(t_g_array)))

#     max_idx = np.unravel_index(np.argmax(results), results.shape)
#     max_value = results[max_idx]
#     omega_d_best = omega_d_array[max_idx[0]]
#     t_g_best = t_g_array[max_idx[1]]

#     print("\n=== Final Results ===")
#     print(f"Best fidelity: {max_value}")
#     print(f"Found at omega_d = {omega_d_best}, t_g = {t_g_best}")
#     print(f"Indices in results array: {max_idx}")

## Parallel Execution

In [6]:
from joblib import Parallel, delayed
from tqdm.notebook import tqdm  # Better in Jupyter

In [7]:
if __name__ == "__main__":
    evals, _ = fluxonium.eigensys(evals_count=levels)
    omega_d_base = evals[1] - evals[0]

    omega_d_array = np.linspace(omega_d_base - 0.005, omega_d_base + 0.005, 10)
    peak_time_noise = 559.5559555955596  
    t_g_array = np.linspace(0.99 * peak_time_noise, 1.01 * peak_time_noise, 10)
    param_pairs = list(itertools.product(omega_d_array, t_g_array))
    print(f"Total simulations to run: {len(param_pairs)}")

    # Parallel execution using joblib
    results_flat = Parallel(n_jobs=-1)(
        delayed(evolve)(omega_d, t_g)
        for (omega_d, t_g) in tqdm(param_pairs, desc="Running simulations")
    )

    results = np.reshape(results_flat, (len(omega_d_array), len(t_g_array)))

    max_idx = np.unravel_index(np.argmax(results), results.shape)
    max_value = results[max_idx]
    omega_d_best = omega_d_array[max_idx[0]]
    t_g_best = t_g_array[max_idx[1]]

    print("\n=== Final Results ===")
    print(f"Best fidelity: {max_value}")
    print(f"Found at omega_d = {omega_d_best}, t_g = {t_g_best}")
    print(f"Indices in results array: {max_idx}")

Total simulations to run: 100


Running simulations:   0%|          | 0/100 [00:00<?, ?it/s]


=== Final Results ===
Best fidelity: 0.7967951428624448
Found at omega_d = 0.5103175305685398, t_g = 560.1776844351102
Indices in results array: (4, 5)


In [8]:
results

array([[0.52523415, 0.47208124, 0.40853384, 0.35835213, 0.33968728,
        0.3591294 , 0.40938583, 0.47124509, 0.52040035, 0.53740077],
       [0.52931675, 0.43935395, 0.36723689, 0.33970667, 0.36675957,
        0.43817766, 0.52673674, 0.59794726, 0.62373604, 0.59422138],
       [0.46944947, 0.37637306, 0.33973653, 0.37314647, 0.46386489,
        0.57722691, 0.66926976, 0.70387153, 0.66749802, 0.57474744],
       [0.38460934, 0.3398093 , 0.37708648, 0.48206882, 0.61439431,
        0.72280135, 0.7648827 , 0.72408477, 0.61645872, 0.48417066],
       [0.33995977, 0.37787508, 0.48970297, 0.6320822 , 0.74973634,
        0.79679514, 0.75472536, 0.63988576, 0.49718885, 0.382354  ],
       [0.37545328, 0.48565538, 0.62760485, 0.74592811, 0.79459354,
        0.75456979, 0.64127715, 0.49893579, 0.38337161, 0.33993373],
       [0.47095419, 0.60221675, 0.71262013, 0.75922886, 0.7240411 ,
        0.62061359, 0.48917929, 0.38138647, 0.33988767, 0.38105235],
       [0.56069483, 0.65596012, 0.6971820

### Param map rework for windows

In [9]:
def param_map(f, parameters, map_fun=map, dtype=object):
    dims_list = [len(i) for i in parameters]
    total_dim = np.prod(dims_list)

    parameters_prod = tuple(itertools.product(*parameters))

    data = np.empty(total_dim, dtype=dtype)
    for i, d in enumerate(tqdm(map_fun(f, parameters_prod), total=total_dim, desc="Simulating")):
        data[i] = d

    return np.reshape(data, dims_list)

In [10]:
if __name__ == "__main__":
    evals, _ = fluxonium.eigensys(evals_count=levels)
    omega_d_base = evals[1] - evals[0]

    omega_d_array = np.linspace(omega_d_base - 0.005, omega_d_base + 0.005, 10)
    peak_time_noise = 559.5559555955596
    t_g_array = np.linspace(0.99 * peak_time_noise, 1.01 * peak_time_noise, 10)

    def evolve_wrapper(params):
        omega_d, t_g = params
        return evolve(omega_d, t_g)

    results = param_map(
        f=lambda p: evolve_wrapper(p),
        parameters=[omega_d_array, t_g_array],
        map_fun=lambda f, params: Parallel(n_jobs=-1)(delayed(f)(p) for p in params),
    )

    max_idx = np.unravel_index(np.nanargmax(results), results.shape)
    max_value = results[max_idx]
    omega_d_best = omega_d_array[max_idx[0]]
    t_g_best = t_g_array[max_idx[1]]

    print("\n=== Final Results ===")
    print(f"Best fidelity: {max_value}")
    print(f"Found at omega_d = {omega_d_best}, t_g = {t_g_best}")
    print(f"Indices in results array: {max_idx}")


KeyboardInterrupt: 